In [1]:
import pymongo
import pandas as pd
from bs4 import BeautifulSoup
from tabulate import tabulate

# Connect to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["books_reviews_db"]
collection = db["reviews"]

In [2]:
query = {"star_rating": {"$in": [1, 5]}, "helpfulness_ratio": {"$gt": 0.8}}
docs = collection.find(query).limit(5)

docs_list = []
for doc in docs:
    # Clean HTML tags from review text
    review_body_cleaned = BeautifulSoup(doc["review_body"], "html.parser").get_text()
    # Remove stray characters
    review_body_cleaned = review_body_cleaned.replace("\\", "")  
    review_body_cleaned = review_body_cleaned.replace("\"", "")
    
    docs_list.append({
        "Star Rating": doc["star_rating"],
        "Helpfulness Ratio": doc["helpfulness_ratio"],
        "Review Body": review_body_cleaned[:100]  # First 150 char. l
    })
df1 = pd.DataFrame(docs_list)

print("Query 1: Reviews with star_rating of 1 or 5 and helpfulness_ratio > 0.8\n")
print(tabulate(df1, headers='keys', tablefmt='pretty', showindex=False))

Query 1: Reviews with star_rating of 1 or 5 and helpfulness_ratio > 0.8

+-------------+-------------------+------------------------------------------------------------------------------------------------------+
| Star Rating | Helpfulness Ratio |                                             Review Body                                              |
+-------------+-------------------+------------------------------------------------------------------------------------------------------+
|     5.0     |        1.0        | Pop A Yellow Smoke by Chuck Truitt is a book that every person needs to read. It gives the face to f |
|     1.0     |        0.9        | According to the following sources, nearly all the content is copied from other people's work, and i |
|     5.0     |        1.0        | Chuck Truitt has provided a window into the time of Vietnam- There are 3,.. no 4 themes intertwined  |
|     5.0     |        1.0        | I really enjoyed this follow-up to the authors previous b

In [3]:
pipeline = [
    {"$group": {
        "_id": "$star_rating",
        "avg_helpfulness": {"$avg": "$helpfulness_ratio"},
        "count": {"$sum": 1}
    }},
    {"$sort": {"_id": 1}}
]

result = collection.aggregate(pipeline)

# Convert to DataFrame
result_list = []
for row in result:
    result_list.append({
        "Star Rating": row["_id"],
        "Average Helpfulness Ratio": row["avg_helpfulness"],
        "Count": row["count"]
    })

df2 = pd.DataFrame(result_list)

# Print the DataFrame directly
print("Query 2: Aggregation - Average helpfulness per star rating\n")
print(tabulate(df2, headers='keys', tablefmt='pretty', showindex=False))

Query 2: Aggregation - Average helpfulness per star rating

+-------------+---------------------------+---------+
| Star Rating | Average Helpfulness Ratio |  Count  |
+-------------+---------------------------+---------+
|     1.0     |    0.5233822038304833     | 5395.0  |
|     2.0     |    0.5739740250402325     | 3604.0  |
|     3.0     |    0.5955325857592736     | 5173.0  |
|     4.0     |    0.6511808177811049     | 11356.0 |
|     5.0     |    0.6901659795957286     | 35494.0 |
+-------------+---------------------------+---------+


In [57]:
# Compare reviews with star ratings 1, 3, and 5
# Compute both:
#   - average helpfulness ratio
#   - average review length
# along with review counts

pipeline = [
    {"$match": {"star_rating": {"$in": [1, 3, 5]}}},
    {"$group": {
        "_id": "$star_rating",
        "avg_helpfulness": {"$avg": "$helpfulness_ratio"},
        "avg_length": {"$avg": "$review_length"},
        "n": {"$sum": 1}
    }},
    {"$sort": {"_id": 1}}
]

cursor = collection.aggregate(pipeline)

docs_list = []
for doc in cursor:
    docs_list.append({
        "Star Rating": doc["_id"],
        "Avg Helpfulness": round(doc["avg_helpfulness"], 2),  # Average helpfulness_ratio
        "Avg Review Length": round(doc["avg_length"], 2),  # Average review_length
        "Count": doc["n"]  # Veri sayısı
    })

df3 = pd.DataFrame(docs_list)

print("Query 3: Average helpfulness and review length for each star rating (1, 3, 5)")
print(tabulate(df3, headers='keys', tablefmt='pretty', showindex=False))


Query 3: Average helpfulness and review length for each star rating (1, 3, 5)
+-------------+-----------------+-------------------+---------+
| Star Rating | Avg Helpfulness | Avg Review Length |  Count  |
+-------------+-----------------+-------------------+---------+
|     1.0     |      0.52       |       837.5       | 3627.0  |
|     3.0     |       0.6       |      1054.06      | 3481.0  |
|     5.0     |      0.69       |      777.17       | 23858.0 |
+-------------+-----------------+-------------------+---------+


In [4]:
# Trend over time
# Group reviews by year and star rating
# Then compute the average helpfulness ratio
# We see if helpfulness changes over years

pipeline = [
    # Match only reviews with a valid year format (starting with 19xx or 20xx)
    {"$match": {"review_date": {"$regex": "^(19|20)"}}},
    # Extract year from review_date string
    {"$project": {
        "year": {"$substr": ["$review_date", 0, 4]},
        "star_rating": 1,
        "helpfulness_ratio": 1
    }},

    # Group by year and star rating, compute average helpfulness
    {"$group": {
        "_id": {"year": "$year", "star_rating": "$star_rating"},
        "avg_helpfulness": {"$avg": "$helpfulness_ratio"}
    }},
    {"$sort": {"_id.year": 1}}
]

result = collection.aggregate(pipeline)

# Convert to df
result_list = []
for row in result:
    result_list.append({
        "Year": row["_id"]["year"],
        "Star Rating": row["_id"]["star_rating"],
        "Average Helpfulness Ratio": row["avg_helpfulness"]
    })

df4 = pd.DataFrame(result_list)

print("Query 4: Average helpfulness per star rating grouped by year\n")
print(tabulate(df4, headers='keys', tablefmt='pretty', showindex=False))

Query 4: Average helpfulness per star rating grouped by year

+------+-------------+---------------------------+
| Year | Star Rating | Average Helpfulness Ratio |
+------+-------------+---------------------------+
| 1936 |     5.0     |           0.75            |
| 1995 |     5.0     |            1.0            |
| 1996 |     4.0     |            0.5            |
| 1996 |     5.0     |    0.5147988869213256     |
| 1997 |     1.0     |            0.0            |
| 1997 |     4.0     |    0.5198809523809523     |
| 1997 |     2.0     |    0.42227272727272724    |
| 1997 |     5.0     |    0.4073694766713908     |
| 1997 |     3.0     |    0.32727272727272727    |
| 1998 |     1.0     |    0.2140276941601445     |
| 1998 |     4.0     |    0.4480897004869881     |
| 1998 |     2.0     |    0.42484126984126985    |
| 1998 |     3.0     |    0.49132874922348607    |
| 1998 |     5.0     |    0.5384104296362366     |
| 1999 |     2.0     |    0.46031270068312025    |
| 1999 |     4.0    